#Natural Language Processing with Disaster Tweets

In [1]:
import kaggle

OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

In [ ]:
! mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
import kaggle

In [7]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 96.8MB/s]


In [8]:
!unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


In [11]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [13]:
train_df.shape

(7613, 5)

In [14]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [15]:
# Fill missing values in 'keyword' and 'location' columns with a placeholder value
train_df['keyword'].fillna('unknown_keyword', inplace=True)
train_df['location'].fillna('unknown_location', inplace=True)

In [16]:
train_df.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

## Preprocessing function

In [17]:
def preprocess_text(text):
  text = text.lower()  # convert text to lowercase
  text = re.sub(r'https?://\S+|www\.\S+','',text) # remove URLs
  text = re.sub(r'[^a-zA-Z\s]','',text) # Remove special characters and punctuation

  tokens = nltk.word_tokenize(text) # Tokenization
  stop_words = set(stopwords.words('english')) # Remove stop words
  filtered_tokens = [word for word in tokens if word not in stop_words]

  text = ' '.join(filtered_tokens)
  return text

In [18]:
# Apply preprocessing to be the 'text' column
train_df['text_cleaned'] = train_df['text'].apply(preprocess_text)

In [19]:
# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_df['text_cleaned'],train_df['target'],test_size=0.2,random_state=42)

In [20]:
#TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

In [21]:
model = LogisticRegression()
model.fit(X_train_tfidf,y_train)

LogisticRegression()

In [22]:
# Make Predictions on the validation data
y_pred_log = model.predict(X_val_tfidf)

In [23]:
#Evaluate the model
print('Validation Accuracy:', accuracy_score(y_val,y_pred_log))
print('Classification Report:')
print(classification_report(y_val,y_pred_log))


Validation Accuracy: 0.8010505581089954
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       874
           1       0.82      0.68      0.75       649

    accuracy                           0.80      1523
   macro avg       0.81      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



Using Test Dataset

In [28]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [29]:
test_df.shape

(3263, 4)

In [30]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [31]:
test_df.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [32]:
# Fill missing values in 'keyword' and 'location' columns with a placeholder value
test_df['keyword'].fillna('unknown_keyword', inplace=True)
test_df['location'].fillna('unknown_location', inplace=True)

In [33]:
test_df.isnull().sum()

id          0
keyword     0
location    0
text        0
dtype: int64

In [34]:
test_df['text_cleaned'] = test_df['text'].apply(preprocess_text)

In [35]:
#TF-IDF vectorization for text data
test_tfidf = tfidf_vectorizer.transform(test_df['text_cleaned'])

In [36]:
test_pred = model.predict(test_tfidf)

In [37]:
submission_df = pd.DataFrame({'id': test_df['id'],'target':test_pred})

In [38]:
submission_df.to_csv('/content/drive/MyDrive/Kaggle Comptetions/submission.csv',index=False)